In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [230]:
import pandas as pd
import numpy as np

items = pd.read_csv('/content/drive/MyDrive/practice/cafe_chatbot/Item_to_id.csv',sep=",",on_bad_lines='skip', encoding='latin-1')
data = pd.read_csv('/content/drive/MyDrive/practice/cafe_chatbot/conversationo.csv',sep=",",on_bad_lines='skip', encoding='latin-1')
food = pd.read_csv('/content/drive/MyDrive/practice/cafe_chatbot/food.csv',sep=",",on_bad_lines='skip', encoding='latin-1')

food = food.rename(columns={'times_appeared':'time','food_rating':'rating'})

In [231]:
#items.head()
#data.head()
food.head()

,id,time,rating
0,1001.0,9.0,2.0
1,1002.0,116.0,2.0
2,1003.0,13.0,2.0
3,1004.0,41.0,2.0
4,1005.0,68.0,1.0


In [232]:
data.dropna(how='any', inplace=True)
data = data.drop_duplicates(['Question','answer'])
data = data.drop([0])
data = data.reset_index()
data.isna().sum()

index       0
Question    0
answer      0
dtype: int64

In [233]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     228 non-null    int64 
 1   Question  228 non-null    object
 2   answer    228 non-null    object
dtypes: int64(1), object(2)
memory usage: 5.5+ KB


In [234]:
data.head()

,index,Question,answer
0,1,do u have coffee,"Yes sir Simple Coffee ,Cappuchino, Americano,..."
1,2,i will take one espresso and 5 americano,Sir thanks for your order. You have ordered 1 ...
2,3,anything special,"We have coffe,pastries,puff pastries and milks..."
3,4,suggest something,"We have coffe,pastries,puff pastries and milks..."
4,5,"do You have Coffee Cappuchino, Americano, Au L...",Yes we have your choice in our cafe.


In [235]:
food['id'] = food.index + 1
food.dropna(how='any', inplace=True)
food = food.reset_index()
food.isna().sum()

index     0
id        0
time      0
rating    0
dtype: int64

In [236]:
food.head()

,index,id,time,rating
0,0,1,9.0,2.0
1,1,2,116.0,2.0
2,2,3,13.0,2.0
3,3,4,41.0,2.0
4,4,5,68.0,1.0


In [237]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

In [246]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def response(sentence):
  result1 = []

  for i in range(len(data['Question'])):
    sen = (sentence, data['Question'][i])
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sen)

    cos_similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

    result1.append(cos_similar[0][0])

  return print(data['answer'][result1.index(max(result1))])

In [247]:
def order_accept(menu_list):
  id_list = []
  for i in menu_list:
    for j, k in enumerate(items['name']):
      if i == k:
        id_list.append(j+1)
  
  time_list = []
  for i in id_list:
    for j, k in enumerate(food['id']):
      if i == k:
        time_list.append(food['time'][j])
  
  max_time = int(max(time_list)/60) + 1

  return max_time

In [248]:
import random

menu_list = []
flag = True
print("Cafebot: Hi, my name is Cafebot. I will hear your order.")
print("--------------------------------------------------------")
print("1. If you want to see random menu, type 'random'")
print("2. If you want to order, enter 'shs' at the beginning of the sentence and enter the menu only")
print("3. If you want to check your order so far, enter 'nowon'")
print("4. If you want to exit, type 'bye'")
print("--------------------------------------------------------")
while(flag==True):
  print("You: ", end="")
  user_response = input()
  if (user_response == 'bye'):
    flag = False
    print("Cafebot: Your order details are :")
    print("")
    for i in menu_list:
      print(i)
    print("")
    print("Cafebot: Thank you. See you again. Have a good day.")
    print("Cafebot: You just have to wait about ", order_accept(menu_list), " minutes ")
    print("--------------------------------------------------------")
  elif user_response[:3] == 'shs':
    print('Cafebot: Your order has been saved! Have further conversations.')
    temp_res = user_response.replace('shs ','')
    menu_list.append(temp_res)
    print("--------------------------------------------------------")
  elif user_response[:5] == 'nowon':
    print('Cafebot: Your order details are as follows :')
    for i in menu_list:
      print(i)
    print("--------------------------------------------------------")
  elif user_response[:6] == 'random':
    print('Cafebot: recommended Menus are :')
    for i in range(5):
      print(random.choice(items['name']))
    print("--------------------------------------------------------")
  elif user_response.lower() in GREETING_INPUTS:
    print(random.choice(GREETING_RESPONSES))
    print("--------------------------------------------------------")
  else:
    print("Cafebot: ", end="")
    user_response = user_response.lower()
    response(user_response)
    print("--------------------------------------------------------")

Cafebot: Hi, my name is Cafebot. I will hear your order.
--------------------------------------------------------
1. If you want to see random menu, type 'random'
2. If you want to order, enter 'shs' at the beginning of the sentence and enter the menu only
3. If you want to check your order so far, enter 'nowon'
4. If you want to exit, type 'bye'
--------------------------------------------------------
You: hi
hi
--------------------------------------------------------
You: do you have coffee
Cafebot: Yes sir  Simple Coffee ,Cappuchino, Americano, Au Laite ,Au Laite Cappucchino ,Chans Espresso ,Con Panna, Black Coffee, Espresso, Frappe ,French Coffee ,Iced Coffee Late ,Irish Coffee, Latte Macchiato and  Wainans Choco Coffee.What would u like to have?
--------------------------------------------------------
You: special something
Cafebot: We have coffe,pastries,puff pastries and milkshakes.What would you like to have?
--------------------------------------------------------
You: random
